# Academy of Py

The Chief Data Scientist for a city's school district is helping the school board and mayor make strategic decisions regarding future school budgets and priorities. As a first task, she has been asked to analyze the district-wide standardized test results. She is given access to every student's math and reading scores, as well as various information on the schools they attend. Her responsibility is to aggregate the data and showcase obvious trends in school performance. Below is the Analysis done for the district.


## Input files

The script assumes two input files used for this analysis. They need to be .csv files as described below:

* One file contains the school data with the following columns : School ID, name, type, size, budget
* The other file contains student data with the following columns : Student ID, name, gender, grade, school, reading_score, math score


## Sneak peak at the input data

The script asks for the user to input the school file name (or relative path) and the student file name (or relative path). It reads them and then proceeds with the analysis

In [1]:
# Import Dependencies
import pandas as pd
import numpy as np

In [4]:
# Read the input files
school_data_file = input("Enter the name of the school data file: ")
student_data_file = input("Enter the name of the student data file: ")

Enter the name of the school data file: raw_data/schools_complete.csv
Enter the name of the student data file: raw_data/students_complete.csv


In [5]:
school_df = pd.read_csv(school_data_file)
student_df = pd.read_csv(student_data_file)

### School Data File

In [6]:
# Check the school data
school_df.head()

,School ID,name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [7]:
school_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 5 columns):
School ID    15 non-null int64
name         15 non-null object
type         15 non-null object
size         15 non-null int64
budget       15 non-null int64
dtypes: int64(3), object(2)
memory usage: 680.0+ bytes


### Student Data File

In [8]:
# Check the student data
student_df.head()

,Student ID,name,gender,grade,school,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [9]:
student_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39170 entries, 0 to 39169
Data columns (total 7 columns):
Student ID       39170 non-null int64
name             39170 non-null object
gender           39170 non-null object
grade            39170 non-null object
school           39170 non-null object
reading_score    39170 non-null int64
math_score       39170 non-null int64
dtypes: int64(3), object(4)
memory usage: 2.1+ MB


## District Summary

High level snapshot (in table form) of the district's key metrics, including:

* Total Schools
* Total Students
* Total Budget
* Average Math Score
* Average Reading Score
* % Passing Math
* % Passing Reading
* Overall Passing Rate (Average of the above two)

In [10]:
# Calculate the totals
school_df.head()
num_schools = len(school_df)
num_students = school_df['size'].sum()
total_budget = school_df['budget'].sum()

# Calculate average scores
avg_math_score = student_df['math_score'].mean()
avg_read_score = student_df['reading_score'].mean()

# Calculate passing rates
num_pass_math = len(student_df[student_df['math_score']>69])
num_pass_read = len(student_df[student_df['reading_score']>69])

percent_pass_math = (num_pass_math/num_students)*100
percent_pass_read = (num_pass_read/num_students)*100

overall_pass = (percent_pass_math + percent_pass_read)/2

# Arrange the numbers in a table for display
snapshot_df = pd.DataFrame({'Total Schools': [num_schools],
                            'Total Students': [num_students],
                            'Total Budget': [total_budget],
                            'Average Math Score': [avg_math_score],
                            'Average Reading Score': [avg_read_score],
                            '% Passing Math': [percent_pass_math],
                            '% Passing Reading': [percent_pass_read],
                            'Overall Passing Rate': [overall_pass]
                           })
snapshot_df = snapshot_df[['Total Schools','Total Students','Total Budget','Average Math Score',
                          'Average Reading Score','% Passing Math','% Passing Reading','Overall Passing Rate']]
snapshot_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,80.393158


## School Summary

Overview table that summarizes key metrics about each school, including:

* School Name
* School Type
* Total Students
* Total School Budget
* Per Student Budget
* Average Math Score
* Average Reading Score
* % Passing Math
* % Passing Reading
* Overall Passing Rate (Average of the above two)


In [11]:
# Create a new data frame for the School Summary, start with the school data and rename the columns
school_summary_df = school_df
school_summary_df.rename(columns={'name':'School Name', 'type':'School Type', 
                                  'size':'Total Students', 'budget':'Total School Budget'}, inplace=True)
school_summary_df.drop('School ID', axis=1, inplace=True)
school_summary_df['Per Student Budget'] = school_summary_df['Total School Budget']/school_summary_df['Total Students']

# Now summarize the student data by school. Create a group by school
student_group = student_df.groupby('school', as_index=False)

# Get the Average scores and rename columns
student_group_df = student_group.mean()[['school','math_score', 'reading_score']]
student_group_df.rename(columns={'math_score':'Average Math Score', 'reading_score':'Average Reading Score'}, inplace=True)

# Get the count of students per school to be used to calculate the passing rates later
count_group_df = student_group.count()[['school', 'Student ID']]
student_group_df = pd.merge(student_group_df, count_group_df, on = 'school')
student_group_df.rename(columns={'Student ID':'Student Count'}, inplace=True)

# Get the count of students grouped by school, who passed math. Use Student Count to get the % Passing Math
math_pass_group_df = student_df[student_df['math_score']>69].groupby('school', as_index=False).count()[['school', 'Student ID']]
student_group_df = pd.merge(student_group_df, math_pass_group_df, on = 'school')
student_group_df.rename(columns={'Student ID':'% Passing Math'}, inplace=True)
student_group_df['% Passing Math'] = (student_group_df['% Passing Math']/student_group_df['Student Count'])*100

# Get the count of students grouped by school, who passed reading. Use Student Count to get the % Passing Reading
reading_pass_group_df = student_df[student_df['reading_score']>69].groupby('school', as_index=False).count()[['school', 'Student ID']]
student_group_df = pd.merge(student_group_df, reading_pass_group_df, on = 'school')
student_group_df.rename(columns={'Student ID':'% Passing Reading'}, inplace=True)
student_group_df['% Passing Reading'] = (student_group_df['% Passing Reading']/student_group_df['Student Count'])*100

# Get the overall passing value as the average of the two rates
student_group_df['Overall Passing Rate'] = (student_group_df['% Passing Reading'] + student_group_df['% Passing Math'])/2

# format all the columns
student_group_df['Average Math Score'] = student_group_df['Average Math Score'].map("{0:.2f}".format)
student_group_df['Average Reading Score'] = student_group_df['Average Reading Score'].map("{0:.2f}".format)
student_group_df['% Passing Math'] = student_group_df['% Passing Math'].map("{0:.2f}".format)
student_group_df['% Passing Reading'] = student_group_df['% Passing Reading'].map("{0:.2f}".format)
student_group_df['Overall Passing Rate'] = student_group_df['Overall Passing Rate'].map("{0:.2f}".format)

#Drop the Student Count as we don't need it anymore
student_group_df.drop('Student Count', axis=1, inplace=True)

#Rename the school column as we now need to merge with the school_summary_df on the School Name column
student_group_df.rename(columns={'school':'School Name'}, inplace=True)

school_summary_df = pd.merge(school_summary_df, student_group_df, on='School Name')

school_summary_df

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,Huang High School,District,2917,1910635,655.0,76.63,81.18,65.68,81.32,73.50
1,Figueroa High School,District,2949,1884411,639.0,76.71,81.16,65.99,80.74,73.36
2,Shelton High School,Charter,1761,1056600,600.0,83.36,83.73,93.87,95.85,94.86
3,Hernandez High School,District,4635,3022020,652.0,77.29,80.93,66.75,80.86,73.81
4,Griffin High School,Charter,1468,917500,625.0,83.35,83.82,93.39,97.14,95.27
5,Wilson High School,Charter,2283,1319574,578.0,83.27,83.99,93.87,96.54,95.20
6,Cabrera High School,Charter,1858,1081356,582.0,83.06,83.98,94.13,97.04,95.59
7,Bailey High School,District,4976,3124928,628.0,77.05,81.03,66.68,81.93,74.31
8,Holden High School,Charter,427,248087,581.0,83.80,83.81,92.51,96.25,94.38
9,Pena High School,Charter,962,585858,609.0,83.84,84.04,94.59,95.95,95.27


## Top Performing Schools (By Passing Rate)

This table highlights the top 5 performing schools based on Overall Passing Rate. Include:

* School Name
* School Type
* Total Students
* Total School Budget
* Per Student Budget
* Average Math Score
* Average Reading Score
* % Passing Math
* % Passing Reading
* Overall Passing Rate (Average of the above two)


In [12]:
# sort the above school summary table by ascending values of Overall Passing Rate
top_5_schools_df = school_summary_df.sort_values('Overall Passing Rate', ascending=False)

# Reset the index and get the first 5 rows
top_5_schools_df.reset_index(inplace=True, drop=True)
top_5_schools_df = top_5_schools_df.iloc[:5, :]
top_5_schools_df

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,Cabrera High School,Charter,1858,1081356,582.0,83.06,83.98,94.13,97.04,95.59
1,Thomas High School,Charter,1635,1043130,638.0,83.42,83.85,93.27,97.31,95.29
2,Griffin High School,Charter,1468,917500,625.0,83.35,83.82,93.39,97.14,95.27
3,Pena High School,Charter,962,585858,609.0,83.84,84.04,94.59,95.95,95.27
4,Wilson High School,Charter,2283,1319574,578.0,83.27,83.99,93.87,96.54,95.20


## Lowest Performing Schools (By Passing Rate)

This table highlights the bottom 5 performing schools based on Overall Passing Rate. Include:

* School Name
* School Type
* Total Students
* Total School Budget
* Per Student Budget
* Average Math Score
* Average Reading Score
* % Passing Math
* % Passing Reading
* Overall Passing Rate (Average of the above two)



In [13]:
# sort the above school summary table by ascending values of Overall Passing Rate
bot_5_schools_df = school_summary_df.sort_values('Overall Passing Rate')

# Reset the index and get the first 5 rows
bot_5_schools_df.reset_index(inplace=True, drop=True)
bot_5_schools_df = bot_5_schools_df.iloc[:5, :]
bot_5_schools_df

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,Rodriguez High School,District,3999,2547363,637.0,76.84,80.74,66.37,80.22,73.29
1,Figueroa High School,District,2949,1884411,639.0,76.71,81.16,65.99,80.74,73.36
2,Huang High School,District,2917,1910635,655.0,76.63,81.18,65.68,81.32,73.50
3,Johnson High School,District,4761,3094650,650.0,77.07,80.97,66.06,81.22,73.64
4,Ford High School,District,2739,1763916,644.0,77.10,80.75,68.31,79.30,73.80


## Math Scores by Grade

This table lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.


In [14]:
# Group the studen data by school and then by grade and get the average of the math_score
grade_group = student_df.groupby(['school', 'grade'], as_index=False)
grade_group_df = grade_group.mean()[['school', 'grade', 'math_score']]
grade_group_df['math_score'] = grade_group_df['math_score'].map("{0:.2f}".format)
grade_group_df.rename(columns={'math_score': 'Avg Math Score'}, inplace=True)

grade_group_df = grade_group_df.pivot(index='school', columns='grade', values='Avg Math Score')
grade_group_df

grade,10th,11th,12th,9th
school,,,,
Bailey High School,77.00,77.52,76.49,77.08
Cabrera High School,83.15,82.77,83.28,83.09
Figueroa High School,76.54,76.88,77.15,76.40
Ford High School,77.67,76.92,76.18,77.36
Griffin High School,84.23,83.84,83.36,82.04
Hernandez High School,77.34,77.14,77.19,77.44
Holden High School,83.43,85.00,82.86,83.79
Huang High School,75.91,76.45,77.23,77.03
Johnson High School,76.69,77.49,76.86,77.19


## Reading Scores by Grade

This table lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.


In [15]:
# Group the studen data by school and then by grade and get the average of the reading_score
grade_group = student_df.groupby(['school', 'grade'], as_index=False)
grade_group_df = grade_group.mean()[['school', 'grade', 'reading_score']]
grade_group_df['reading_score'] = grade_group_df['reading_score'].map("{0:.2f}".format)
grade_group_df.rename(columns={'reading_score': 'Avg Reading Score'}, inplace=True)

grade_group_df = grade_group_df.pivot(index='school', columns='grade', values='Avg Reading Score')
grade_group_df

grade,10th,11th,12th,9th
school,,,,
Bailey High School,80.91,80.95,80.91,81.30
Cabrera High School,84.25,83.79,84.29,83.68
Figueroa High School,81.41,80.64,81.38,81.20
Ford High School,81.26,80.40,80.66,80.63
Griffin High School,83.71,84.29,84.01,83.37
Hernandez High School,80.66,81.40,80.86,80.87
Holden High School,83.32,83.82,84.70,83.68
Huang High School,81.51,81.42,80.31,81.29
Johnson High School,80.77,80.62,81.23,81.26


## Scores by School Spending

This table breaks down school performances based on average Spending Ranges (Per Student). We've used 4 reasonable bins to group school spending. Included in the table are each of the following:

* Average Math Score
* Average Reading Score
* % Passing Math
* % Passing Reading
* Overall Passing Rate (Average of the above two)


In [16]:
min_student_budget = int(school_summary_df['Per Student Budget'].min())
max_student_budget = int(school_summary_df['Per Student Budget'].max())

binsize = int(round((max_student_budget-min_student_budget)/4))+1
bud_bins = np.arange(min_student_budget-1, max_student_budget+binsize, binsize)

# Create the labels for the bins
bud_labels = []

for i in range(len(bud_bins)):
    bud_labels.append(str(bud_bins[i-1]) + "-" + str(bud_bins[i]))
    
# We remove the first label as it isn't needed 
bud_labels.pop(0)


'657-577'

In [17]:
# Create a new data frame for the School Summary, start with the school data and rename the columns
school_summary_df = school_df
school_summary_df['Spending Range Per Student'] = pd.cut(school_summary_df['Per Student Budget'], bins=bud_bins, labels=bud_labels)

school_summary_df = school_summary_df[['School Name', 'Spending Range Per Student']]
school_summary_df

# merge the bins into the students table
student_df.rename(columns={'school':'School Name'}, inplace=True)
students_bins_df = pd.merge(student_df, school_summary_df, on='School Name')
students_bins_df

# Now summarize the student data by spending bins
student_bins_group = students_bins_df.groupby('Spending Range Per Student', as_index=False)

scores_by_spending_df = student_bins_group.count()[['Spending Range Per Student','Student ID']]
avgs_by_spending_df = student_bins_group.mean()[['Spending Range Per Student','reading_score', 'math_score']]

scores_by_spending_df = pd.merge(scores_by_spending_df, avgs_by_spending_df, on = 'Spending Range Per Student')
scores_by_spending_df.rename(columns={'Student ID':'Student Count',
                                      'reading_score':'Avg Reading Score',
                                     'math_score':'Avg Math Score'}, inplace=True)

# Get the count of students grouped by spending bins, who passed math. Use Student Count to get the % Passing Math
math_pass_group_df = students_bins_df[students_bins_df['math_score']>69].groupby('Spending Range Per Student', as_index=False).count()[['Spending Range Per Student', 'Student ID']]
scores_by_spending_df = pd.merge(scores_by_spending_df, math_pass_group_df, on='Spending Range Per Student')
scores_by_spending_df.rename(columns={'Student ID':'% Passing Math'}, inplace=True)
scores_by_spending_df['% Passing Math'] = (scores_by_spending_df['% Passing Math']/scores_by_spending_df['Student Count'])*100

# Get the count of students grouped by spending bins, who passed math. Use Student Count to get the % Passing Math
read_pass_group_df = students_bins_df[students_bins_df['reading_score']>69].groupby('Spending Range Per Student', as_index=False).count()[['Spending Range Per Student', 'Student ID']]
scores_by_spending_df = pd.merge(scores_by_spending_df, read_pass_group_df, on='Spending Range Per Student')
scores_by_spending_df.rename(columns={'Student ID':'% Passing Reading'}, inplace=True)
scores_by_spending_df['% Passing Reading'] = (scores_by_spending_df['% Passing Reading']/scores_by_spending_df['Student Count'])*100

# Get the overall passing value as the average of the two rates
scores_by_spending_df['Overall Passing Rate'] = (scores_by_spending_df['% Passing Reading'] + scores_by_spending_df['% Passing Math'])/2

#Drop the Student Count as we don't need it anymore
scores_by_spending_df.drop('Student Count', axis=1, inplace=True)

# format all the columns
scores_by_spending_df['Avg Math Score'] = scores_by_spending_df['Avg Math Score'].map("{0:.2f}".format)
scores_by_spending_df['Avg Reading Score'] = scores_by_spending_df['Avg Reading Score'].map("{0:.2f}".format)
scores_by_spending_df['% Passing Math'] = scores_by_spending_df['% Passing Math'].map("{0:.2f}".format)
scores_by_spending_df['% Passing Reading'] = scores_by_spending_df['% Passing Reading'].map("{0:.2f}".format)
scores_by_spending_df['Overall Passing Rate'] = scores_by_spending_df['Overall Passing Rate'].map("{0:.2f}".format)

scores_by_spending_df

,Spending Range Per Student,Avg Reading Score,Avg Math Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,577-597,83.96,83.36,93.70,96.69,95.19
1,597-617,83.84,83.53,94.12,95.89,95.01
2,617-637,81.31,77.86,70.32,83.41,76.86
3,637-657,81.23,77.54,68.74,82.15,75.44


## Scores by School Size

This table breaks down school performances based on a reasonable approximation of school size (Small, Medium, Large). We've used 4 reasonable bins to group school size. Included in the table are each of the following:

* Average Math Score
* Average Reading Score
* % Passing Math
* % Passing Reading
* Overall Passing Rate (Average of the above two)


In [18]:
min_student_count = int(school_df['Total Students'].min())
max_student_count = int(school_df['Total Students'].max())

# bins
binsize = int(round((max_student_count-min_student_count)/3))+1
size_bins = np.arange(min_student_count-1, max_student_count+binsize, binsize)

# labels for the bins
size_labels = ['Small', 'Medium', 'Large']

# Create a new data frame for the School Summary, start with the school data and rename the columns
school_summary_df = school_df
school_summary_df['School Size'] = pd.cut(school_summary_df['Total Students'], bins=size_bins, labels=size_labels)

school_summary_df = school_summary_df[['School Name', 'School Size']]
school_summary_df

# merge the bins into the students table
student_df.rename(columns={'school':'School Name'}, inplace=True)
students_bins_df = pd.merge(student_df, school_summary_df, on='School Name')

# Now summarize the student data by School Size
student_bins_group = students_bins_df.groupby('School Size', as_index=False)

# Get average scores
scores_by_size_df = student_bins_group.count()[['School Size','Student ID']]
avgs_by_size_df = student_bins_group.mean()[['School Size','reading_score', 'math_score']]
scores_by_size_df = pd.merge(scores_by_size_df, avgs_by_size_df, on = 'School Size')
scores_by_size_df.rename(columns={'Student ID':'Student Count',
                                      'reading_score':'Avg Reading Score',
                                     'math_score':'Avg Math Score'}, inplace=True)

# Get the count of students grouped by School Size, who passed math. Use Student Count to get the % Passing Math
math_pass_group_df = students_bins_df[students_bins_df['math_score']>69].groupby('School Size', as_index=False).count()[['School Size', 'Student ID']]
scores_by_size_df = pd.merge(scores_by_size_df, math_pass_group_df, on='School Size')
scores_by_size_df.rename(columns={'Student ID':'% Passing Math'}, inplace=True)
scores_by_size_df['% Passing Math'] = (scores_by_size_df['% Passing Math']/scores_by_size_df['Student Count'])*100

# Get the count of students grouped by School Size, who passed reading. Use Student Count to get the % Passing Math
read_pass_group_df = students_bins_df[students_bins_df['reading_score']>69].groupby('School Size', as_index=False).count()[['School Size', 'Student ID']]
scores_by_size_df = pd.merge(scores_by_size_df, read_pass_group_df, on='School Size')
scores_by_size_df.rename(columns={'Student ID':'% Passing Reading'}, inplace=True)
scores_by_size_df['% Passing Reading'] = (scores_by_size_df['% Passing Reading']/scores_by_size_df['Student Count'])*100

# Get the overall passing value as the average of the two rates
scores_by_size_df['Overall Passing Rate'] = (scores_by_size_df['% Passing Reading'] + scores_by_size_df['% Passing Math'])/2

#Drop the Student Count as we don't need it anymore
scores_by_size_df.drop('Student Count', axis=1, inplace=True)

# format all the columns
scores_by_size_df['Avg Math Score'] = scores_by_size_df['Avg Math Score'].map("{0:.2f}".format)
scores_by_size_df['Avg Reading Score'] = scores_by_size_df['Avg Reading Score'].map("{0:.2f}".format)
scores_by_size_df['% Passing Math'] = scores_by_size_df['% Passing Math'].map("{0:.2f}".format)
scores_by_size_df['% Passing Reading'] = scores_by_size_df['% Passing Reading'].map("{0:.2f}".format)
scores_by_size_df['Overall Passing Rate'] = scores_by_size_df['Overall Passing Rate'].map("{0:.2f}".format)

scores_by_size_df

,School Size,Avg Reading Score,Avg Math Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,Small,83.88,83.44,93.66,96.67,95.17
1,Medium,81.65,78.16,72.34,83.84,78.09
2,Large,80.93,77.07,66.47,81.11,73.79


## Scores by School Types

This table breaks down school performances based on school type (Charter vs. District).Included in the table are each of the following:

* Average Math Score
* Average Reading Score
* % Passing Math
* % Passing Reading
* Overall Passing Rate (Average of the above two)



In [19]:
# Add the school district info to the students table and drop unwanted columns
students_df = pd.merge(student_df, school_df[['School Name', 'School Type']], on='School Name')
students_df.drop(['name','gender','grade'], axis=1, inplace=True)

# Now summarize the student data by School Type
student_group = students_df.groupby('School Type', as_index=False)

# Get the Average scores and counts by School Type
scores_by_type_df = student_group.count()[['School Type','Student ID']]
avgs_by_type_df = student_group.mean()[['School Type','reading_score', 'math_score']]
scores_by_type_df = pd.merge(scores_by_type_df, avgs_by_type_df, on = 'School Type')
scores_by_type_df.rename(columns={'Student ID':'Student Count',
                                      'reading_score':'Avg Reading Score',
                                     'math_score':'Avg Math Score'}, inplace=True)

# Get the count of students grouped by School Type, who passed math. Use Student Count to get the % Passing Math
math_pass_group_df = students_df[students_df['math_score']>69].groupby('School Type', as_index=False).count()[['School Type', 'Student ID']]
scores_by_type_df = pd.merge(scores_by_type_df, math_pass_group_df, on='School Type')
scores_by_type_df.rename(columns={'Student ID':'% Passing Math'}, inplace=True)
scores_by_type_df['% Passing Math'] = (scores_by_type_df['% Passing Math']/scores_by_type_df['Student Count'])*100

# Get the count of students grouped by School Type, who passed reading. Use Student Count to get the % Passing Math
read_pass_group_df = students_df[students_df['reading_score']>69].groupby('School Type', as_index=False).count()[['School Type', 'Student ID']]
scores_by_type_df = pd.merge(scores_by_type_df, read_pass_group_df, on='School Type')
scores_by_type_df.rename(columns={'Student ID':'% Passing Reading'}, inplace=True)
scores_by_type_df['% Passing Reading'] = (scores_by_type_df['% Passing Reading']/scores_by_type_df['Student Count'])*100

# Get the overall passing value as the average of the two rates
scores_by_type_df['Overall Passing Rate'] = (scores_by_type_df['% Passing Reading'] + scores_by_type_df['% Passing Math'])/2

#Drop the Student Count as we don't need it anymore
scores_by_type_df.drop('Student Count', axis=1, inplace=True)

# format all the columns
scores_by_type_df['Avg Math Score'] = scores_by_type_df['Avg Math Score'].map("{0:.2f}".format)
scores_by_type_df['Avg Reading Score'] = scores_by_type_df['Avg Reading Score'].map("{0:.2f}".format)
scores_by_type_df['% Passing Math'] = scores_by_type_df['% Passing Math'].map("{0:.2f}".format)
scores_by_type_df['% Passing Reading'] = scores_by_type_df['% Passing Reading'].map("{0:.2f}".format)
scores_by_type_df['Overall Passing Rate'] = scores_by_type_df['Overall Passing Rate'].map("{0:.2f}".format)

scores_by_type_df

,School Type,Avg Reading Score,Avg Math Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,Charter,83.90,83.41,93.70,96.65,95.17
1,District,80.96,76.99,66.52,80.91,73.71
